In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy.optimize import leastsq
from scipy import ndimage
from allfunc import *
import random
from astropy.table import Table, hstack
import os
import glob
from astropy.table import Table, vstack, join
import pandas as pd

In [2]:
def popt_central_fit(wo_wave,central_data,central_error,obj,p_init,z,broad2=False,MC_loops=100):
    k = 1+z
    select = (wo_wave>4750*k) & (wo_wave < 5090*k)
    if broad2:
        full_gauss = full_gauss2
    else:
        full_gauss = full_gauss1
    popt_full_fit,pcov_full_fit = leastsq(full_gauss,x0=p_init,args=(wo_wave[select],central_data[select],central_error[select]),maxfev = 10000000)
    parameters_MC = np.zeros((len(popt_full_fit),MC_loops))
    for l in range(MC_loops):
        iteration_data = np.random.normal(central_data[select],central_error[select]) 
        popt_MC,pcov_MC =leastsq(full_gauss,x0=p_init,args=(wo_wave[select],iteration_data,central_error[select]),maxfev = 10000000)
        parameters_MC[:,l]=popt_MC
        parameters_err = np.std(parameters_MC,1)  
    (amp_Hb_error,amp_OIII5007_error,vel_OIII_error,vel_sigma_OIII_error,amp_Hb_br_error,amp_OIII5007_br_error,vel_OIII_br_error,vel_sigma_OIII_br_error,amp_Hb1_error,amp_Fe5018_1_error,vel_Hb1_error,vel_sigma_Hb1_error,amp_Hb2_error,amp_Fe5018_2_error,vel_Hb2_error,vel_sigma_Hb2_error,m_error,c_error) = parameters_err
    return popt_full_fit,parameters_err 

def popt_arb_fit(wave,data,error,p_init,broad2=False,MC_loops=100):
    if broad2:
        full_gauss = full_gauss2
    else:
        full_gauss = full_gauss1
    popt2,pcov2 = leastsq(full_gauss,x0=p_init,args=(wave,data,error),maxfev = 10000000)
    parameters_MC = np.zeros((len(popt2),MC_loops))
    for l in range(MC_loops):
        iteration_data = np.random.normal(data,error) 
        popt_MC,pcov_MC =leastsq(full_gauss,x0=p_init,args=(wave,iteration_data,error),maxfev = 10000000)
        parameters_MC[:,l]=popt_MC
        parameters_err = np.std(parameters_MC,1)  
    (amp_Hb_error,amp_OIII5007_error,vel_OIII_error,vel_sigma_OIII_error,amp_Hb_br_error,amp_OIII5007_br_error,vel_OIII_br_error,vel_sigma_OIII_br_error,amp_Hb1_error,amp_Fe5018_1_error,vel_Hb1_error,vel_sigma_Hb1_error,amp_Hb2_error,amp_Fe5018_2_error,vel_Hb2_error,vel_sigma_Hb2_error,m_error,c_error) = parameters_err
    return popt2,parameters_err

def arb_plot(obj,wave,data,error,popt,arb_fit,arb_residual):
    (amp_Hb_fit,amp_OIII5007_fit,vel_OIII_fit,vel_sigma_OIII_fit,amp_Hb_br_fit,amp_OIII5007_br_fit,vel_OIII_br_fit,vel_sigma_OIII_br_fit,amp_Hb1_fit,amp_Fe5018_1_fit,vel_Hb1_fit,vel_sigma_Hb1_fit,amp_Hb2_fit,amp_Fe5018_2_fit,vel_Hb2_fit,vel_sigma_Hb2_fit,m_fit,c_fit) = popt
    plt.plot(wave,data,'k-',label='data')
    plt.plot(wave,arb_fit,'r-',label='fit')
    plt.plot(wave,arb_residual,label='residual')
    plt.plot(wave,Hb_O3_gauss(wave,amp_Hb_fit,amp_OIII5007_fit,vel_OIII_fit,vel_sigma_OIII_fit) + continuum(wave,m_fit,c_fit),'-g',label='core')
    plt.plot(wave,Hb_O3_gauss(wave,amp_Hb_br_fit,amp_OIII5007_br_fit,vel_OIII_br_fit,vel_sigma_OIII_br_fit) + continuum(wave,m_fit,c_fit),color ='purple',label='wing')
    plt.plot(wave,Hb_Fe_doublet_gauss(wave,amp_Hb1_fit,amp_Fe5018_1_fit,vel_Hb1_fit,vel_sigma_Hb1_fit) + continuum(wave,m_fit,c_fit),color ='brown',label='BLR1')
    plt.plot(wave,Hb_Fe_doublet_gauss(wave,amp_Hb2_fit,amp_Fe5018_2_fit,vel_Hb2_fit,vel_sigma_Hb2_fit) + continuum(wave,m_fit,c_fit),'-c',label='BLR2')
    plt.title('%s_brightest_pixel_fit'%(obj))
    plt.xlabel(r"Wavelength ($\AA$)")
    plt.ylabel(r"Flux Density ($\times 10^{-16}$ erg s$^{-1}$ cm$^{-2}$ Hz$^{-1}$) ")
    plt.legend()
    plt.show() 
    


In [3]:
def arb_spectroastrometry(arb_data,arb_error,wave,fixed_param,broad2=False,MC_loops=100):
    output_par = np.zeros((10,arb_data.shape[1],arb_data.shape[2]),dtype=np.float32)
    output_par_err = np.zeros((10,arb_data.shape[1],arb_data.shape[2]),dtype=np.float32)
    if broad2:
        full_gauss_fixkin = full_gauss2_fixkin
    else:
        full_gauss_fixkin = full_gauss1_fixkin
    for i in range(arb_data.shape[1]):
        for j in range(arb_data.shape[2]):
            (spectrum,error) = (arb_data[:,i,j],arb_error[:,i,j])    
            popt1,pcov1 = leastsq(full_gauss_fixkin,x0=[0.1, 0.1, 0.1, 0.1, 0.01, 0.01, 0.01, 0.01, -0.7, 0.001],args=(wave,spectrum,error,fixed_param),maxfev = 10000000)
            
            output_par[:,i,j]=popt1
        
            spec_parameters_MC = np.zeros((len(popt1),MC_loops))
            for l in range(MC_loops):
                iteration_data = np.random.normal(spectrum,error)   
                popt_spec_MC,pcov_spec_MC = leastsq(full_gauss_fixkin,x0=[0.1, 0.1, 0.1, 0.1, 0.01, 0.01, 0.01, 0.01, -0.7, 0.001],args=(wave,iteration_data,error,fixed_param),maxfev = 10000000)
                spec_parameters_MC[:,l]=popt_spec_MC
                
            spec_parameters_err = np.std(spec_parameters_MC,1)
            output_par_err[:,i,j] = spec_parameters_err
    return output_par,output_par_err

def flux_map(spec_par,obj,x1,y1,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    hdus=[]
    hdus.append(fits.PrimaryHDU())
    hdus.append(fits.ImageHDU(spec_par[0,:,:],name='amp_Hb'))
    hdus.append(fits.ImageHDU(spec_par[1,:,:],name='amp_OIII5007'))
    hdus.append(fits.ImageHDU(spec_par[2,:,:],name='amp_OIII5007_br'))
    hdus.append(fits.ImageHDU(spec_par[3,:,:],name='amp_Hb_br'))
    hdus.append(fits.ImageHDU(spec_par[4,:,:],name='amp_Hb1'))
    hdus.append(fits.ImageHDU(spec_par[5,:,:],name='amp_Hb2'))
    hdus.append(fits.ImageHDU(spec_par[6,:,:],name='amp_Fe5018_1'))
    hdus.append(fits.ImageHDU(spec_par[7,:,:],name='amp_Fe5018_2'))
    hdus.append(fits.ImageHDU(spec_par[8,:,:],name='m'))
    hdus.append(fits.ImageHDU(spec_par[9,:,:],name='c'))
    hdu = fits.HDUList(hdus)
    hdu.writeto('%s/%s/Systematic Error/x%sy%s_artificial_fluxmap_%s.fits'%(destination_path_cube,obj,x1,y1,obj),overwrite='True')

def flux_map_err(spec_err,obj,x1,y1,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    hdus=[]
    hdus.append(fits.PrimaryHDU())
    hdus.append(fits.ImageHDU(spec_err[0,:,:],name='amp_Hb_err'))
    hdus.append(fits.ImageHDU(spec_err[1,:,:],name='amp_OIII5007_err'))
    hdus.append(fits.ImageHDU(spec_err[2,:,:],name='amp_OIII5007_br_err'))
    hdus.append(fits.ImageHDU(spec_err[3,:,:],name='amp_Hb_br_err'))
    hdus.append(fits.ImageHDU(spec_err[4,:,:],name='amp_Hb1_err'))
    hdus.append(fits.ImageHDU(spec_err[5,:,:],name='amp_Hb2_err'))
    hdus.append(fits.ImageHDU(spec_err[6,:,:],name='amp_Fe5018_1_err'))
    hdus.append(fits.ImageHDU(spec_err[7,:,:],name='amp_Fe5018_2_err'))
    hdus.append(fits.ImageHDU(spec_err[8,:,:],name='m_err'))
    hdus.append(fits.ImageHDU(spec_err[9,:,:],name='c_err'))
    hdu = fits.HDUList(hdus)
    hdu.writeto('%s/%s/Systematic Error/x%sy%s_artificial_fluxerrormap_%s.fits'%(destination_path_cube,obj,x1,y1,obj),overwrite='True')
    

  

In [4]:
def remove_bad_pixel(data,err):
    data[data == 0.01] = 0
    err[err < 10**(-14)] =10**(10)  
    return data,err

def ellip_moffat2D(p,xy,data,error):
    xy=(x,y)
    [amp, x0, y0, A, B, C, alpha] = p 
    g = amp*((1.0+A*(x-x0)**2+B*(y-y0)**2+C*(x-x0)*(y-y0))**(-alpha))
    h = (data - g)/error
    return h.ravel()

def ellip_moffat2D_fixkin(p,xy,data,error,fixed_param):
    [A,B,C,alpha] = fixed_param
    xy=(x,y)
    [amp, x0, y0] = p    
    g = amp*((1.0+(A*((x-x0)**2))+(B*((y-y0)**2))+(C*(x-x0)*(y-y0)))**(-alpha))
    h = (data - g)/error
    return h.ravel()

def arb_moffat_fit(data,error,box_size,amp,x0,y0,muse_sampling_size,fix_par,MC_loops=100):
    y, x = np.mgrid[:box_size, :box_size]
    xy=(x,y)
    
    if fix_par is None:
        p= [amp,x0,y0,0.1,0.1,-0.00008,3]
        popt,pcov = leastsq(ellip_moffat2D,x0=p,args=(xy,data,error),maxfev = 10000000)
        [amp_out,x0_out,y0_out,A_out,B_out,C_out,alpha_out]= popt
        theta_out = 0.5*np.arctan(C_out/(A_out - B_out))
        a_out = np.sqrt(2/(A_out + B_out + np.sqrt(C_out**2 +(A_out - B_out)**2)))
        b_out = np.sqrt(2/(A_out + B_out - np.sqrt(C_out**2 +(A_out - B_out)**2)))
        [fwhm1_out,fwhm2_out] = [2*a_out*np.sqrt(2**(1/alpha_out)-1),2*b_out*np.sqrt(2**(1/alpha_out)-1)]
        par = [amp_out,x0_out,y0_out,A_out,B_out,C_out,alpha_out,a_out,b_out,theta_out,fwhm1_out,fwhm2_out]
        mod_rav = data.ravel() - ellip_moffat2D(popt,xy,data,error)*error.ravel()
    else:
        p= [amp,x0,y0]
        popt,pcov = leastsq(ellip_moffat2D_fixkin,x0=p,args=(xy,data,error,fix_par),maxfev = 10000000)
        [amp_out,x0_out,y0_out]= popt
        par = popt
        mod_rav = data.ravel() - ellip_moffat2D_fixkin(popt,xy,data,error,fix_par)*error.ravel()
    model = mod_rav.reshape(box_size,box_size)
    residual = data-model
    res = (residual/error)

    parameters_MC = np.zeros((len(par),MC_loops))
    for l in range(MC_loops):
        iteration_data = np.random.normal(data,error) 
        if fix_par is None:
            p= [amp,x0,y0,0.1,0.1,-0.00008,3]
            popt_MC,pcov_MC = leastsq(ellip_moffat2D,x0=p,args=(xy,iteration_data,error),maxfev = 10000000)
            [amp_MC,x0_MC,y0_MC,A_MC,B_MC,C_MC,alpha_MC]= popt_MC
            theta_MC = 0.5*np.arctan(C_MC/(A_MC - B_MC))
            a_MC = np.sqrt(2/(A_MC + B_MC + np.sqrt(C_MC**2 +(A_MC - B_MC)**2)))
            b_MC = np.sqrt(2/(A_MC + B_MC - np.sqrt(C_MC**2 +(A_MC - B_MC)**2)))
            [fwhm1_MC,fwhm2_MC] = [2*a_MC*np.sqrt(2**(1/alpha_MC)-1),2*b_MC*np.sqrt(2**(1/alpha_MC)-1)]
            par_MC = [amp_MC,x0_MC,y0_MC,A_MC,B_MC,C_MC,alpha_MC,a_MC,b_MC,theta_MC,fwhm1_MC,fwhm2_MC]   
            parameters_MC[:,l] = par_MC    
        else:
            p= [amp,x0,y0]
            popt_MC,pcov_MC = leastsq(ellip_moffat2D_fixkin,x0=p,args=(xy,iteration_data,error,fix_par),maxfev = 10000000)
            [amp_MC,x0_out,y0_out]= popt_MC
            parameters_MC[:,l] = popt_MC    
    parameters_err = np.std(parameters_MC,1) 
    return par,parameters_err

def arb_moffat_table(obj,x1,y1,Hb_x,Hb_y,OIII_br_x,OIII_br_y,OIII_nr_x,OIII_nr_y,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    full_data = [x1,y1,Hb_x,Hb_y,OIII_br_x,OIII_br_y,OIII_nr_x,OIII_nr_y]
    column_names={'x_cor':0,'y_cor':1,'Hb_x':2,'Hb_y':3,'OIII_br_x':4,'OIII_br_y':5,'OIII_nr_x':6,'OIII_nr_y':7}
    columns=[]
    for key in column_names.keys():
        columns.append(fits.Column(name=key,format='E',array=[full_data[column_names[key]]]))
    coldefs = fits.ColDefs(columns)
    hdu = fits.BinTableHDU.from_columns(coldefs)
    hdu.writeto('%s/%s/Systematic Error/x%sy%s_artificial_moffatcentertab_%s.fits'%(destination_path_cube,obj,x1,y1,obj),overwrite=True)

In [5]:
def algorithm_script(obj,z,broad2,p_init,x1,y1,prefix_path_cube="/home/mainak/ftp.hidrive.strato.com/users/login-carsftp/IFU_data",destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE",MC_loops=100,box_size=4):
    (wo_cube,wo_err,wo_wave,wo_header) = loadCube('%s/%s/%s.wo_absorption.fits'%(destination_path_cube,obj,obj)) 
    
   ############################ Retrieve the central error spectrum ########################
    [x0,y0] = agn_location(obj)
    [brightest_pixel_x,brightest_pixel_y] = [int(x0),int(y0)]
    (central_spec,central_err_spec) = (wo_cube[:,brightest_pixel_y,brightest_pixel_x],wo_err[:,brightest_pixel_y,brightest_pixel_x])
    k=1+z
    select = (wo_wave>5030*k) & (wo_wave<5050*k)
    cont_data = central_spec[select]
    S = np.mean(cont_data)
    N = np.std(cont_data)
    S_N = (S/N)
    #print (S_N,S)
    
    muse_sampling_size = arb_sampling_size(wo_cube)  
    #print (muse_sampling_size)
    
    ###################### Recreate the central spectrum###################################################
    (amp_Hb,amp_OIII5007,vel_OIII,vel_sigma_OIII,amp_Hb_br,amp_OIII5007_br,vel_OIII_br,vel_sigma_OIII_br,amp_Hb1,amp_Fe5018_1,vel_Hb1,vel_sigma_Hb1,amp_Hb2,amp_Fe5018_2,vel_Hb2,vel_sigma_Hb2,m,c) = full_par_central(obj)
    popt_mod = [amp_Hb,amp_OIII5007,vel_OIII,vel_sigma_OIII,amp_Hb_br,amp_OIII5007_br,vel_OIII_br,vel_sigma_OIII_br,amp_Hb1,amp_Fe5018_1,vel_Hb1,vel_sigma_Hb1,amp_Hb2,amp_Fe5018_2,vel_Hb2,vel_sigma_Hb2,m,c]
    fit = central_spectrum(wo_wave,popt_mod,z,broad2)
    
    ############################### get the psf from the moffat model###########################################
    (amp_Hb_blr,x0_Hb_blr,y0_Hb_blr,A_Hb_blr,B_Hb_blr,C_Hb_blr,alpha_Hb_blr) = ellip_moffat_par(obj)
    if muse_sampling_size == 0.4:
        (x0_Hb_blr,y0_Hb_blr) = (x0_Hb_blr-8,y0_Hb_blr-8)
    else:
        (x0_Hb_blr,y0_Hb_blr) = (x0_Hb_blr-15,y0_Hb_blr-15)
    #(amp_OIII_br,x0_OIII_br,y0_OIII_br,amp_OIII_nr,x0_OIII_nr,y0_OIII_nr) = moffat_O3_par(obj)
    psf = model(amp_Hb_blr,x0_Hb_blr-np.random.random(1),y0_Hb_blr-np.random.random(1)- 0.5,A_Hb_blr,B_Hb_blr,C_Hb_blr,alpha_Hb_blr,muse_sampling_size)
    #hdu = fits.PrimaryHDU(psf)
    #hdu.writeto('/home/mainak/psf_test.fits',overwrite=True)
    
    ############################### Load the 3" minicube #######################################################
    (mini_cube,mini_err,mini_wave,mini_header) = loadCube('%s/%s/%s.3_arcsec_minicube.fits'%(destination_path_cube,obj,obj))    
    
    ############################### Create the artificial source ##########################################
    #scale_SN(fit,ref_flux,ref_SN)
    (cube,err) = artf_source(psf,fit,S,S_N)
    
    ########################## feeding the 'good coordinates' to the pipeline########################################
    try:
        #[[x_act,y_act]] = test(z,wo_cube,wo_wave,brightest_pixel_x,brightest_pixel_y,x1,y1)
        (x_act,y_act) = (x1,y1)
        #print ('arbitray coordinates are',(x_act,y_act))
        (x_l,x_r,y_l,y_r) = new_cube_cor(x_act,y_act,muse_sampling_size)
        #print (x_l,x_r,y_l,y_r)
        (arb_data,arb_error,wave) = new_minicube(z,wo_wave,wo_cube,wo_err,cube,err,x_l,x_r,y_l,y_r)
        box_size = arb_data.shape[1]
      
     ################### central fit#######################################################  
        (x0,y0) = arb_brightest_pixel(arb_data)
        (data,error) = (arb_data[:,y0,x0],arb_error[:,y0,x0])
        (popt,par_err) = popt_arb_fit(wave,data,error,p_init,broad2,MC_loops=100)
        (arb_fit,arb_residual) = arb_fit_and_residual(wave,data,error,popt,broad2)
       # arb_plot(obj,wave,data,error,popt,arb_fit,arb_residual)   
        
     ################## around center fit###################################################        
        (amp_Hb_fit,amp_OIII5007_fit,vel_OIII_fit,vel_sigma_OIII_fit,amp_Hb_br_fit,amp_OIII5007_br_fit,vel_OIII_br_fit,vel_sigma_OIII_br_fit,amp_Hb1_fit,amp_Fe5018_1_fit,vel_Hb1_fit,vel_sigma_Hb1_fit,amp_Hb2_fit,amp_Fe5018_2_fit,vel_Hb2_fit,vel_sigma_Hb2_fit,m_fit,c_fit) = popt
        fixed_param = [vel_OIII_fit,vel_sigma_OIII_fit,vel_OIII_br_fit,vel_sigma_OIII_br_fit,vel_Hb1_fit,vel_sigma_Hb1_fit,vel_Hb2_fit,vel_sigma_Hb2_fit]  
        #fixed_param = [vel_OIII,vel_sigma_OIII,vel_OIII_br,vel_sigma_OIII_br,vel_Hb1,vel_sigma_Hb1,vel_Hb2,vel_sigma_Hb2]  
        (spec_par,spec_err) = arb_spectroastrometry(arb_data,arb_error,wave,fixed_param,broad2,MC_loops=100)
        flux_map(spec_par,obj,x1,y1)
        flux_map_err(spec_err,obj,x1,y1)
        
    ################ Data, error and amplitude from light profiles #############################################################
        (Hb_blr_br_dat,OIII_br_dat,OIII_nr_dat,Hb_blr_br_error,OIII_br_error,OIII_nr_error) = flux_data_and_error(obj,x1,y1)
    
    ######## Remove any bad pixel ##############################   
        (Hb_blr_br_data,Hb_blr_br_err) = remove_bad_pixel(Hb_blr_br_dat,Hb_blr_br_error)
        (OIII_br_data,OIII_br_err) = remove_bad_pixel(OIII_br_dat,OIII_br_error)
        (OIII_nr_data,OIII_nr_err) = remove_bad_pixel(OIII_nr_dat,OIII_nr_error)  
    
    ######## Set up all initial position guesses #######################
        box_size = np.shape(Hb_blr_br_data)[0]
        (amp_Hb_blr_br,amp_OIII_br,amp_OIII_nr) = (np.max(Hb_blr_br_data),np.max(OIII_br_data),np.max(OIII_nr_data))
        (brightest_pixel_Hb_blr_br_y,brightest_pixel_Hb_blr_br_x) = ndimage.measurements.maximum_position(Hb_blr_br_data)
        (brightest_pixel_OIII_br_y,brightest_pixel_OIII_br_x) = ndimage.measurements.maximum_position(OIII_br_data)
        (brightest_pixel_OIII_nr_y,brightest_pixel_OIII_nr_x) = ndimage.measurements.maximum_position(OIII_nr_data)
    
     ########### Moffat modeling part ######################   
        (Hb_par,Hb_error) = arb_moffat_fit(Hb_blr_br_data,Hb_blr_br_err,box_size,amp_Hb_blr_br,brightest_pixel_Hb_blr_br_x,brightest_pixel_Hb_blr_br_y,muse_sampling_size,None,100)
        fixed_param = [Hb_par[3],Hb_par[4],Hb_par[5],Hb_par[6]]
        (OIII_br_par,OIII_br_error) = arb_moffat_fit(OIII_br_data,OIII_br_err,box_size,amp_OIII_br,brightest_pixel_OIII_br_x,brightest_pixel_OIII_br_y,muse_sampling_size,fixed_param,100)   
        (OIII_nr_par,OIII_nr_error) = arb_moffat_fit(OIII_nr_data,OIII_nr_err,box_size,amp_OIII_nr,brightest_pixel_OIII_nr_x,brightest_pixel_OIII_nr_y,muse_sampling_size,fixed_param,100)   
    
    ############# offset##############################################################
        (Hb_x,Hb_y) = (Hb_par[1],Hb_par[2])
        (OIII_br_x,OIII_br_y) = (OIII_br_par[1],OIII_br_par[2])
        (OIII_nr_x,OIII_nr_y) = (OIII_nr_par[1],OIII_nr_par[2])
        print (OIII_br_x - Hb_x,OIII_br_y - Hb_y)
        arb_moffat_table(obj,x1,y1,Hb_x,Hb_y,OIII_br_x,OIII_br_y,OIII_nr_x,OIII_nr_y)
    except (RuntimeError,RuntimeWarning):# (TypeError,IndexError):
        pass 

In [ ]:
z = {"HE0021-1810":0.05352,"HE0021-1819":0.053197,"HE0040-1105":0.041692,"HE0108-4743":0.02392,"HE0114-0015":0.04560
    ,"HE0119-0118":0.054341,"HE0212-0059":0.026385,"HE0224-2834":0.059800,"HE0227-0913":0.016451,"HE0232-0900":0.043143
    ,"HE0253-1641":0.031588,"HE0345+0056":0.031,"HE0351+0240":0.036,"HE0412-0803":0.038160,"HE0429-0247":0.042009
    ,"HE0433-1028":0.035550,"HE0853+0102":0.052,"HE0934+0119":0.050338,"HE1011-0403":0.058314,"HE1017-0305":0.049986
    ,"HE1029-1831":0.040261,"HE1107-0813":0.058,"HE1108-2813":0.024013,"HE1126-0407":0.061960,"HE1237-0504":0.009
    ,"HE1248-1356":0.01465,"HE1330-1013":0.022145,"HE1353-1917":0.035021,"HE1417-0909":0.044,"HE2128-0221":0.05248
    ,"HE2211-3903":0.039714,"HE2222-0026":0.059114,"HE2233+0124":0.056482,"HE2302-0857":0.046860}

objs = z.keys()

broad2= {'HE0021-1810':False,'HE0021-1819':False,'HE0040-1105':False,'HE0108-4743':True,'HE0114-0015':False
        ,'HE0119-0118':True,'HE0212-0059':True,'HE0224-2834':False,'HE0227-0913':True,'HE0232-0900':False
        ,'HE0253-1641':True,'HE0345+0056':True,'HE0351+0240':True,'HE0412-0803':False,'HE0429-0247':True
        ,'HE0433-1028':True,'HE0853+0102':True,'HE0934+0119':True,'HE1011-0403':True,'HE1017-0305':False
        ,'HE1029-1831':True,'HE1107-0813':True,'HE1108-2813':False,'HE1126-0407':True,'HE1237-0504':False
        ,'HE1248-1356':False,'HE1330-1013':True,'HE1353-1917':True,'HE1417-0909':False,'HE2128-0221':False
        ,'HE2211-3903':False,'HE2222-0026':True,'HE2233+0124':True,'HE2302-0857':True}

p_init= {'HE0021-1810':[0.1,1.5248,16056,50.0,1.0,3.0,15956,100.0,1.0,1.0,16056,1000.0,0,0,16056,1000.0,-0.001,2.0]
        ,'HE0021-1819':[1,12.5,15959,50.0,3,3,15759,200.0,2,2,15959,1000.0,0,0,15959,1000.0,-0.001,0.1]
        ,'HE0040-1105':[1.139,1.5,12507.0,50.0,1.0,3.0,12307.0,200.0,1.0,1.0,12507,1000.0,0,0,12507,1000.0,-0.001,2.0]
        ,'HE0108-4743':[1.7,4.5,7160.0,40.0,0.12,2.5,7100.0,120.0,1.0,1.0,6960,1300.0,1.0,1.0,7190,430.0,-0.001,2.0]
        ,'HE0114-0015':[0.1,1.5,13680,50.0,1.0,3.0,13480,100.0,1.0,1.0,13680,1000.0,0,0,13680,1000.0,-0.001,0.3]
        ,'HE0119-0118':[1,12.5,16302,50.0,3,3,16102,100.0,2,2,16302,1000.0,1,1,16302,2500.0,-0.001,0.1]
        ,'HE0212-0059':[0.1,1.0,7930,170.0,0.1,0.1,7630,450.0,1.0,1.0,7620,4000.0,0.1,0.1,4650,1090.0,-0.001,2.0]
        ,'HE0224-2834':[0.2,2.5,17940,50.0,0.3,0.3,17640,200.0,0.1,0.1,17940,1500.0,0,0,17940,1500.0,-0.001,0.1]
        ,'HE0227-0913':[3,6,4785,150,2,7,4385,300.0,7,1.8,4935,400.0,3,0.5,4735,1500.0,-0.3,4.7]
        ,'HE0232-0900':[2,40,12942,50.0,5,5,12742,100.0,5,1,12942,1000.0,0,0,12942,1000.0,-0.001,0.1]                                     
        ,'HE0253-1641':[2,10,9476,110.0,1,6,9176,370.0,1,1,9476,1500.0,0.5,0.5,9476,550.0,-0.001,0.5]
        ,'HE0345+0056':[0.5,20,9300,150,5,10,9000,300,9,1.8,9300,400,30,2.5,9300,1500,0.3,7.1]
        ,'HE0351+0240':[2.2,37,10600,77,1.1,1.5,10618,277,1.5,0.07,11580,320,3,0.35,10650,1344,-0.3,3]                                                    
        ,'HE0412-0803':[1.139,1.5,11448,50.0,0.1,0.3,11248,100.0,0.1,0.1,11448,1000.0,0,0,11448,1000.0,-0.001,0.5]
        ,'HE0429-0247':[0.9,8.5,12600,90,0.2,2.0,12500,220,1.7,0.07,12600,450,1.3,0.2,12600,1600,-0.5,4.0]
        ,'HE0433-1028':[0.9,4.0,10665.0,100.0,1.0,1.0,10465.0,300.0,1.0,1.0,10665,800.0,0.1,0.1,10665,1500.0,-0.1,1.0]
        ,'HE0853+0102':[0.1,0.9,15700,110.0,0.02,0.3,15670,320.0,0.1,0.1,15900,1640.0,0.0,0.0,17250,810.0,-0.001,0.1]
        ,'HE0934+0119':[0.6,1.9,15101,100.0,0.2,0.2,14901,400.0,0.3,0.1,15101,1000.0,0.1,0.1,15101,1000.0,0.001,0.7]
        ,'HE1011-0403':[0.1,0.6,17494,60.0,0.2,0.2,17244,200.0,0.3,0.1,17494,1500.0,0.1,0.1,17494,1500.0,0.001,0.7]
        ,'HE1017-0305':[0.1,1.5,14695,50,0.08,0.4,14595,200,0.7,0.09,14695,2000,0,0,0,0,-0.3,2.0]
        ,'HE1029-1831':[0.6,1.3,12114,50.0,0.2,0.2,11978,200.0,0.3,0.1,12114,1000.0,0.2,0.2,12114,1000.0,-0.001,0.2]
        ,'HE1107-0813':[0.04,0.3,17500,100,0.2,0.13,17400,480,0.9,0.13,17500,1000,0.9,0.08,16800,2000,-0.9,8]
        ,'HE1108-2813':[1.139,1.5,7200.0,50.0,1.0,3.0,7000.0,100.0,1.0,1.0,7200,1000.0,0,0,7200,1000.0,-0.001,2.0]
        ,'HE1126-0407':[1,13,18200,100,2.5,3,18000,300,10,1.3,18200,800,7,1.8,18200,1600,-0.9,15]
        ,'HE1237-0504':[1,10,2700.0,80.0,0.5,2,2700.0,200,2,0.5,2700.0,1500.0,0,0,2700.0,1000.0,-0.001,0.002]
        ,'HE1248-1356':[0.1,1.5,4395.0,50.0,1.0,3.0,4195.0,100.0,1.0,1.0,4395,1000.0,0,0,0,0.0,-0.001,2.0]
        ,'HE1330-1013':[0.15,0.6,6643,90,0.05,0.15,6543,200,0.12,0.04,6643,1500,0.3,0.04,6643,500,-0.02,0.3]
        ,'HE1353-1917':[0.07,0.7,10490.0,80.0,0.39,0.05,10306.0,490.0,0.1,0.02,8600.0,1500.0,0.14,0.02,12326.0,1500.0,-0.001,0.002]
        ,'HE1417-0909':[1,12.5,13200,50.0,3,3,13000,100.0,2,2,13200,1000.0,0,0,13200,1000.0,-0.001,0.1]
        ,'HE2128-0221':[0.1,1.5,15744,50.0,1.0,3.0,15544,100.0,1.0,1.0,15744,1000.0,0,0,15744,1000.0,-0.001,2.0]
        ,'HE2211-3903':[0.6,2.4,11914,50.0,0.1,0.1,11714,100.0,0.1,0.1,11914,200.0,0,0,11914,100.0,-0.001,0.2]
        ,'HE2222-0026':[0.08,0.4,17400.0,140.0,0.04,0.05,17150.0,300.0,0.4,0.1,18500,650.0,0.02,0.01,17460,1750.0,-0.001,0.002]
        ,'HE2233+0124':[0.1,1.2,16944.0,100.0,1.0,3.0,17044.0,300.0,1.0,1.0,16944,1200.0,1.0,1.0,16944,4000.0,-0.001,2.0]
        ,'HE2302-0857':[1,8,14058,200,0.1,2,14258,300,1,0.1,14058,1000,0.1,0.1,14058,1000,0.01,0.2]}
    

for obj in objs:
    print ('%s'%(obj))
    destination_path_cube='/home/mainak/Downloads/Outflow_paper1/MUSE'
    hdu = fits.open('%s/%s/Systematic Error/%s_cord_table.fits'%(destination_path_cube,obj,obj))
    central_tab = hdu[1].data
    number_of_entry = central_tab.field('No_iteration')
    hdu.close()
    (mini_cube,mini_err,mini_wave,mini_header) = loadCube('%s/%s/%s.3_arcsec_minicube.fits'%(destination_path_cube,obj,obj))    
    box_size = np.shape(mini_cube)[1]  
    y, x = np.mgrid[:box_size, :box_size]
    xy=(x,y)
    for i in range(len(number_of_entry)):
        x1 = int(central_tab.field('x_cor')[i])
        y1 = int(central_tab.field('y_cor')[i])
        algorithm_script(obj,z[obj],broad2[obj],p_init[obj],x1,y1) 
        
    input_filename1='./*moffatcentertab_%s.fits'%(obj)
    input_filename2='./*fluxmap_%s.fits'%(obj)
    input_filename3='./*fluxerrormap_%s.fits'%(obj)

    output_filename='%s_syst_table_new.fits'%(obj)

    dir_name = '%s/%s/MUSE Systematic Error New'%(destination_path_cube,obj)   
    os.chdir(dir_name)
    infiles1 = sorted(glob.glob(input_filename1)) # './*source.fits'
    infiles2 = sorted(glob.glob(input_filename2)) # './*source.fits'
    infiles3 = sorted(glob.glob(input_filename3)) # './*source.fits'

    tabledict = {}
    
    for i, file in enumerate(infiles1):
        hdulist = fits.open(file)
        bintab = hdulist[1].data
        table = Table(bintab)
        name = table['Hb_x'][0]  
        tabledict[name] = table

    tab = vstack(list(tabledict.values()))
    df = tab.to_pandas()
    df = df.sort_values(by='Hb_x', ascending=True)
    dtab = Table.from_pandas(df)
    dtab.write(output_filename, format='fits', overwrite=True)
    for i, file in enumerate(infiles1):
         os.remove(file)
    for i, file in enumerate(infiles2):
         os.remove(file)
    for i, file in enumerate(infiles3):
         os.remove(file)
